<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/TEI_XML%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%92%E7%B8%A6%E6%9B%B8%E3%81%8DPDF%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEI/XMLファイルを縦書きPDFに変換する

In [ ]:
# !rm -rf kouigenjimonogatari
# !rm -rf data


## Install Quarto

In [ ]:
QUARTO_VERSION="1.2.174"

In [ ]:
!mkdir -p /opt/quarto/${QUARTO_VERSION}
url = f"https://github.com/quarto-dev/quarto-cli/releases/download/v{QUARTO_VERSION}/quarto-{QUARTO_VERSION}-linux-amd64.tar.gz"
!curl -o quarto.tar.gz -L $url
!tar -zxf quarto.tar.gz \
    -C "/opt/quarto/${QUARTO_VERSION}" \
    --strip-components=1
!/opt/quarto/"${QUARTO_VERSION}"/bin/quarto check
!ln -s /opt/quarto/${QUARTO_VERSION}/bin/quarto /usr/local/bin/quarto

## 校異源氏物語テキストのダウンロード

In [ ]:
!git clone --depth 1 https://github.com/kouigenjimonogatari/kouigenjimonogatari.github.io kouigenjimonogatari

## TEI/XMLファイルの内容をqmdファイルに変換する

In [ ]:
import glob
files = glob.glob("./kouigenjimonogatari/tei/*.xml")
files.sort()
file = files[0]

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open(file,'r'), "xml")
elements = soup.findChildren(text=True, recursive=True)

import os

id = os.path.splitext(os.path.basename(file))[0]

title = soup.find("title").text
author = soup.find("author").text

elements = soup.find("body").find("p").findChildren()

text = ""

for e in elements:

    if e.name == "pb":
        text += "\n"

    if e.name == "seg":
        text += e.text + "  \n"

opath = f"data/{id}.qmd"
os.makedirs(os.path.dirname(opath), exist_ok=True)

text = f"""---
title: "{title}"
author: "{author}"
format: 
    docx:
        reference-doc: /content/kouigenjimonogatari/tools/genji-doc-style.docx
---
{text.strip()}
"""

with open(opath, "w") as f:
    f.write(text)

## qmdファイルをdocxに変換する

In [ ]:
!quarto render data/$id\.qmd --to docx 

`data/01.docx`にWordファイルが出力されます。